In [45]:
caimport argparse
import RetrieveImage as retrieveImage
import os
import csv
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import *
import json
import pandas as pd
import metadata_preprocessing as processing
import xgboost as xgb
import DeepImageUtils as IU
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix, hstack,find
import keras

from keras import optimizers, callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
from keras.layers import Input, Dropout, Dense
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [7]:
data_path = os.path.abspath(globals()['_dh'][0] + '/..') + '/Data'
# images_path = "/ceph/csedu-scratch/project/akaradathodi/Images/"
images_path = os.path.abspath(data_path + '/../../') + '/Images/'

image_features_path = os.path.abspath(globals()['_dh'][0] + '/..') + '/Data/total_features.npz'

In [92]:
test_features = np.load(data_path + "/test_features.npz", allow_pickle=True)
validation_features = np.load(data_path + "/validation_features.npz", allow_pickle=True)
train_features = np.load(data_path + "/train_features.npz", allow_pickle=True)
train_features = train_features['arr_0']
test_features = test_features['arr_0']
validation_features = validation_features['arr_0']


In [53]:
def run_on_xgb_regressor(train_data, y_train, test_data):
    xgb_model = xgb.XGBRegressor()
    xgb_model.fit(train_data, y_train)
    predictions = xgb_model.predict(test_data)
    return predictions

In [126]:
def run_on_ridge_regression(train_data, y_train, test_data):
    ridge_model = Ridge(solver='lsqr', fit_intercept=False)  # solver='lsqr' reduces time to train significantly
    ridge_model.fit(train_data, y_train)

    predictions = ridge_model.predict(test_data)
    return predictions

In [127]:
def run_on_light_bgm_regressor(train_data, y_train, test_data):
    lgbm_model = LGBMRegressor()
    lgbm_model.fit(train_data, y_train)
    predictions = lgbm_model.predict(test_data)
    return predictions

In [97]:
def find_rmse_mae_r2(predicted_price, actual_price):
    predicted_price = np.array(predicted_price_xgb)
    actual_price = np.array(actual_price)

    results = {
            'MSE': mean_squared_error(predicted_price, actual_price),
            'MAE': mean_absolute_error(predicted_price, actual_price),
            'R2': r2_score(predicted_price, actual_price),
        }
    with open(data_path + '/output/feature_stacking_xgb_scores_' + datetime.now().strftime(
                "%Y-%m-%d %H:%M:%S") + '.json', 'w') as file:
            json.dump(results, file)

In [10]:
test_df = pd.read_csv(data_path + "/test_records.csv")
validation_df = pd.read_csv(data_path + "/validation_records.csv")
train_df = pd.read_csv(data_path + "/train_records.csv")
y_train = train_df['price']
y_test = test_df['price']
y_validation = validation_df['price']

In [76]:
# df= pd.DataFrame.from_dict({item: train_features[item] for item in train_features.files}, orient='index')
# df.iloc[0]
train_features = train_features.f.arr_0
test_features = test_features.f.arr_0
validation_features = validation_features.f.arr_0

In [78]:
train_feature = train_features[()].todense()
test_features = test_features[()].todense()
validation_features = validation_features[()].todense()

In [94]:
test_features[()]

<2147x7515 sparse matrix of type '<class 'numpy.float32'>'
	with 3620055 stored elements in Compressed Sparse Row format>

In [95]:
train_features[()]

<6457x7515 sparse matrix of type '<class 'numpy.float32'>'
	with 10744004 stored elements in Compressed Sparse Row format>

In [51]:
y

array([   0,    0,    0, ..., 7514, 7514, 7514], dtype=int32)

In [ ]:
y_train

In [128]:
predicted_price_xgb = []
predicted_price_xgb = (run_on_ridge_regression(train_features[()], y_train, test_features[()]))

In [131]:
predicted_price_xgb[100]

array([-0.6444516], dtype=float32)

In [124]:
y_test[200]

998

In [129]:
find_rmse_mae_r2(predicted_price_xgb, y_test)

In [21]:
y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_val = y_scaler.transform(y_test.values.reshape(-1, 1))

model_in = Input(shape = (7515,), dtype = 'float32', sparse = True)
out = Dense(1024, activation='relu')(model_in)
out = Dropout(0.2)(out)
out = Dense(512, activation='relu')(out)
out = Dropout(0.2)(out)
out = Dense(256, activation='relu')(out)
out = Dense(128, activation='relu')(out)
out = Dense(64, activation='relu')(out)
out = Dense(32, activation='relu')(out)
out = Dense(16, activation='relu')(out)
model_out = Dense(1)(out)    

model = Model(model_in, model_out)
    
model.compile(loss='mean_squared_error', optimizer = tf.keras.optimizers.Adam(learning_rate=0.001))
    
model.summary()
    


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 7515)]            0         
_________________________________________________________________
dense_24 (Dense)             (None, 1024)              7696384   
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_25 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_27 (Dense)             (None, 128)               3289

In [22]:
callbacks = [
    EarlyStopping(monitor = 'val_loss', patience = 3)
]

In [42]:
model.fit(train_features, y_train, batch_size = 1024, epochs = 20, verbose = 1, validation_data = (validation_features, y_validation), callbacks = callbacks)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type csr_matrix).

In [ ]:
predictions = model.predict(X_test)[:, 0]
y_pred = y_scaler.inverse_transform(predictions.reshape(-1, 1))[:, 0]

In [20]:
model, predicted = mlp_regr(train_features, y_train, test_features)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 7515)]            0         
_________________________________________________________________
dense_16 (Dense)             (None, 1024)              7696384   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_19 (Dense)             (None, 128)               3289

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type csr_matrix).